# 1. Title and Problem Statement
Financial professionals, investors, and credit officers must sift through lengthy balance sheets, management discussions, and statistical appendices before they can form a view on a company's fundamentals. The manual process is slow, error-prone, and often leads to missed red flags or opportunities. This notebook demonstrates how AI-assisted reasoning over text prompts and image scans of financial statements can compress the time needed to extract actionable insights.

## 2. Objective of the Project
This project showcases how a conversation-style text prompt ("chat prompt") can be paired with image understanding to read financial statements. Chat prompts tell the AI model which metrics investors care about, while image prompts focus on extracting structured values from scanned pages, tables, and charts. By orchestrating both inputs, the analyst receives explanations, trend narratives, and investor-ready commentary without manually parsing every page.

In [ ]:
# 3. Import Required Libraries
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import Markdown, display

plt.style.use("seaborn-v0_8")

In [ ]:
# 4. Load Financial Document Images
assets_dir = Path("LLM project") / "assets"
image_files = [
    assets_dir / "balance_sheet_sample.png",
    assets_dir / "figures_glance_sample.png",
]

images = []
for path in image_files:
    if not path.exists():
        raise FileNotFoundError(f"Expected image missing: {path}")
    images.append(Image.open(path))

print(f"Loaded {len(images)} financial document images from {assets_dir}.")

## 5. Display and Inspect Images

In [ ]:
fig_captions = [
    "Condensed consolidated balance sheet spanning multiple reporting periods.",
    "Key performance indicators showing profitability, efficiency, and capital ratios.",
]

for idx, (img, caption) in enumerate(zip(images, fig_captions), start=1):
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"Image {idx}: {caption}")
    plt.show()

**Image Summaries**
1. **Image 1** – Balance sheet excerpt detailing assets, liabilities, customer loans, and equity across three reporting dates.
2. **Image 2** – "Figures at a glance" dashboard summarizing profitability (net profit, ROE, EPS), efficiency (cost/income), risk (cost of risk), and capital ratios (CET1, leverage) via bar charts.

## 6. Chat Prompt Design (TEXT PROMPT)
**User Prompt**
> You are an equity research analyst reviewing a newly released annual report. Extract and narrate the most recent revenue, net profit, earnings per share (EPS), and return on equity (ROE) visible across the provided reporting periods. Contrast the latest period against the prior two periods, call out directional trends (growth, decline, stability), and explain drivers like margin shifts or volume changes. Highlight material risks, challenges, or regulatory comments, and close with an investor-oriented takeaway that states whether the financial trajectory supports an overweight, neutral, or underweight stance.

## 7. Image Prompt Design (VISION PROMPT)
**Image Prompt**
> Review each uploaded scan of balance sheets, income statements, and KPI dashboards. Identify every numeric column and row header, transcribe totals such as total assets, customer loans, deposits, and equity, and capture comparative figures for the last three reporting dates. When a table references capital ratios or coverage metrics, extract the percentage along with its label. Summarize whether assets, liabilities, and capital buffers are growing, contracting, or stable, and flag any unusually large quarter-on-quarter movements.

### Chart and Graph Analysis Prompt
**Chart and Graph Analysis Prompt**
> Identify the type of chart (bar chart, line graph, ratio comparison, trend chart) contained in each figure. Read the axis labels, legends, and time periods to anchor every metric. Detect whether values show upward, downward, or stable trajectories, compare performance across quarters or years, and highlight notable spikes, drops, or anomalies. Translate every visual trend into an explicit financial implication for profitability, capital strength, or efficiency.
>
> "Analyze the provided financial charts and graphs. Identify key trends in metrics such as net profit, return on equity, earnings per share, cost/income ratio, CET1 ratio, and leverage ratio. Explain whether each metric shows growth, decline, or stability over time and describe what this indicates about the company’s financial performance."

## 8. Combined Chat + Image + Chart Analysis Simulation
The following simulation assumes an AI system processes:
1. The chat-style prompt that prioritizes investor-grade commentary.
2. The image prompt that focuses on tabular extractions from balance sheets.
3. The chart and graph prompt that decodes KPI trend plots.

To mimic this workflow, we compile structured data (as if parsed from the uploaded scans) and then automatically draft a research-style note that blends numerical deltas with narrative interpretation. The output is structured across the requested sections: Key Financial Metrics, Income and Expenses, Balance Sheet Highlights, Credit Quality, Strategic and Operational Updates, and Market Conditions and Outlook.

In [ ]:
# Simulated structured extraction from images (as if parsed by AI)
financial_metrics = pd.DataFrame(
    {
        "Metric": [
            "Net Profit (EUR mn)",
            "Return on Equity (%)",
            "Earnings per Share (EUR)",
            "Cost/Income Ratio (%)",
            "CET1 Ratio Basel III (%)",
            "CET1 Ratio Basel IV (%)",
            "Leverage Ratio (%)",
        ],
        "Q2 2023": [759, 11.6, 0.73, 71.6, 15.0, 14.8, 5.2],
        "Q4 2023": [642, 11.4, 0.66, 59.2, 13.9, 13.5, 5.3],
        "Q2 2024": [690, 13.1, 0.78, 58.7, 14.1, 13.8, 5.5],
    }
)

income_expenses = pd.DataFrame(
    {
        "Line Item": ["Total Income", "Operating Expenses"],
        "Q2 2023": [15600, 11180],
        "Q4 2023": [14980, 10720],
        "Q2 2024": [16120, 11560],
    }
)

balance_sheet = pd.DataFrame(
    {
        "Line Item": [
            "Customer Loans",
            "Customer Deposits",
            "Total Assets",
            "Total Equity",
        ],
        "30 Sep 2023": [479000, 494200, 1098200, 62300],
        "31 Dec 2023": [483300, 498900, 1113600, 62800],
        "30 Jun 2024": [489800, 503700, 1129400, 63800],
    }
)

credit_quality = pd.DataFrame(
    {
        "Metric": ["Cost of Risk (bps)", "Net Interest Margin (bps)"],
        "Q2 2023": [-4, 152],
        "Q4 2023": [-1, 162],
        "Q2 2024": [2, 165],
    }
)

financial_metrics

In [ ]:
display(Markdown("### Simulated Income and Expense Extraction"))
display(income_expenses)

display(Markdown("### Simulated Balance Sheet Highlights"))
display(balance_sheet)

display(Markdown("### Simulated Credit Quality Metrics"))
display(credit_quality)

In [ ]:
# Helper functions to fetch metric values
latest_period = "Q2 2024"
prior_period = "Q4 2023"
earliest_period = "Q2 2023"


def get_metric(df, metric_name):
    return df.loc[df[df.columns[0]] == metric_name]


def fmt_change(curr, prev):
    change = curr - prev
    direction = "higher" if change > 0 else "lower" if change < 0 else "flat"
    return change, direction

net_profit_row = get_metric(financial_metrics, "Net Profit (EUR mn)")
roe_row = get_metric(financial_metrics, "Return on Equity (%)")
eps_row = get_metric(financial_metrics, "Earnings per Share (EUR)")
ci_row = get_metric(financial_metrics, "Cost/Income Ratio (%)")
lever_row = get_metric(financial_metrics, "Leverage Ratio (%)")

summary_text = f"""
### Integrated AI Financial Summary
**Key Financial Metrics**  
Net profit reached {net_profit_row[latest_period].item():.0f} EUR mn in {latest_period}, {fmt_change(net_profit_row[latest_period].item(), net_profit_row[prior_period].item())[0]:+.0f} vs {prior_period} and {fmt_change(net_profit_row[latest_period].item(), net_profit_row[earliest_period].item())[0]:+.0f} vs {earliest_period}. ROE improved to {roe_row[latest_period].item():.1f}% alongside EPS of {eps_row[latest_period].item():.2f} EUR, signaling profitability momentum.

**Income and Expenses**  
Income expanded to {income_expenses.loc[0, latest_period]:,.0f} while operating costs were {income_expenses.loc[1, latest_period]:,.0f}, pushing the cost-income ratio down to {ci_row[latest_period].item():.1f}%. Efficiency gains stem from revenue resilience outpacing expense creep.

**Balance Sheet Highlights**  
Customer loans climbed to {balance_sheet.loc[0, '30 Jun 2024']:,.0f} and deposits to {balance_sheet.loc[1, '30 Jun 2024']:,.0f}, supporting total assets of {balance_sheet.loc[2, '30 Jun 2024']:,.0f}. Equity now stands at {balance_sheet.loc[3, '30 Jun 2024']:,.0f}, illustrating steady capital accretion.

**Credit Quality**  
Cost of risk normalized to {credit_quality.loc[0, latest_period]:.0f} bps, only mildly above the negative releases booked last year. Net interest margin widened to {credit_quality.loc[1, latest_period]:.0f} bps, aided by loan repricing.

**Strategic and Operational Updates**  
Management commentary inferred from the prompts would emphasize disciplined growth in core lending, cost discipline, and continued digital investment that keeps operating leverage positive. The CET1 Basel IV buffer at {financial_metrics.loc[financial_metrics['Metric']=='CET1 Ratio Basel IV (%)', latest_period].item():.1f}% exceeds the 13.5% target, enabling ongoing distributions.

**Market Conditions and Outlook**  
Trend analysis across charts indicates profitability metrics trending upward while risk indicators remain benign. Given macro uncertainty, guidance likely remains cautious, but current momentum supports a constructive stance so long as credit costs stay contained.
"""

display(Markdown(summary_text))

## 9. Insights, Risks, and Decision Support
- **Actionable Insights**: The most recent quarter shows synchronized improvements across net profit, ROE, and EPS, while capital ratios stay above management's hurdle. This combination supports a constructive stance on capital deployment and shareholder returns.  
- **Risks and Uncertainties**: Compression in customer deposits and a mild uptick in cost of risk suggest sensitivity to funding costs and credit normalization. Macro shocks or higher impairments could quickly erode margins.  
- **AI Impact**: Automating image parsing plus chat-style reasoning reduces manual reading time, surfaces cross-section metrics instantly, and ensures analysts focus on interpretation rather than transcription.

## 10. Conclusion
By orchestrating structured chat prompts with image-specific instructions, this notebook shows how AI can move beyond transcription to deliver synthesized financial narratives. The dual-prompt approach captures raw figures, contextualizes chart dynamics, and converts them into investor-ready commentary—dramatically reducing manual review times while preserving analytical rigor.